---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-text-mining/resources/d9pwm) course resource._

---

# Assignment 1

In this assignment, you'll be working with messy medical data and using regex to extract relevant infromation from the data. 

Each line of the `dates.txt` file corresponds to a medical note. Each note has a date that needs to be extracted, but each date is encoded in one of many formats.

The goal of this assignment is to correctly identify all of the different date variants encoded in this dataset and to properly normalize and sort the dates. 

Here is a list of some of the variants you might encounter in this dataset:
* 04/20/2009; 04/20/09; 4/20/09; 4/3/09
* Mar-20-2009; Mar 20, 2009; March 20, 2009;  Mar. 20, 2009; Mar 20 2009;
* 20 Mar 2009; 20 March 2009; 20 Mar. 2009; 20 March, 2009
* Mar 20th, 2009; Mar 21st, 2009; Mar 22nd, 2009
* Feb 2009; Sep 2009; Oct 2010
* 6/2008; 12/2009
* 2009; 2010

Once you have extracted these date patterns from the text, the next step is to sort them in ascending chronological order accoring to the following rules:
* Assume all dates in xx/xx/xx format are mm/dd/yy
* Assume all dates where year is encoded in only two digits are years from the 1900's (e.g. 1/5/89 is January 5th, 1989)
* If the day is missing (e.g. 9/2009), assume it is the first day of the month (e.g. September 1, 2009).
* If the month is missing (e.g. 2010), assume it is the first of January of that year (e.g. January 1, 2010).
* Watch out for potential typos as this is a raw, real-life derived dataset.

With these rules in mind, find the correct date in each note and return a pandas Series in chronological order of the original Series' indices.

For example if the original series was this:

    0    1999
    1    2010
    2    1978
    3    2015
    4    1985

Your function should return this:

    0    2
    1    4
    2    0
    3    1
    4    3

Your score will be calculated using [Kendall's tau](https://en.wikipedia.org/wiki/Kendall_rank_correlation_coefficient), a correlation measure for ordinal data.

*This function should return a Series of length 500 and dtype int.*

In [2]:
import pandas as pd

doc = []
with open('dates.txt') as file:
    for line in file:
        doc.append(line)

df = pd.Series(doc)
df.head(10)

0         03/25/93 Total time of visit (in minutes):\n
1                       6/18/85 Primary Care Doctor:\n
2    sshe plans to move as of 7/8/71 In-Home Servic...
3                7 on 9/27/75 Audit C Score Current:\n
4    2/6/96 sleep studyPain Treatment Pain Level (N...
5                    .Per 7/06/79 Movement D/O note:\n
6    4, 5/18/78 Patient's thoughts about current su...
7    10/24/89 CPT Code: 90801 - Psychiatric Diagnos...
8                         3/7/86 SOS-10 Total Score:\n
9             (4/10/71)Score-1Audit C Score Current:\n
dtype: object

In [6]:
def date_sorter():
    
    # Your code here
    import re
    all_months = {'jan': '01',
              'feb': '02',
              'mar': '03',
              'apr': '04',
              'may': '05',
              'jun': '06',
              'jul': '07',
              'aug': '08',
              'sep': '09',
              'oct': '10',
              'nov': '11',
              'dec': '12'}

    def cleanup(inp):
        inp = inp.replace('.', ' ')
        inp = inp.replace(',', ' ')
        inp = inp.replace('-', ' ')
        inp = inp.replace('th', '')
        inp = inp.replace('st', '')
        inp = inp.replace('nd', '')
        inp = inp.lower()
        return inp

    # 04/20/2009; 04/20/09; 4/20/09; 4/3/09 
    p1 = r'(\d{1,2}[/-]\d{1,2}[/|-]\d{2,4})'
    def f1(inp):
        inp = inp.replace('-', '/')
        month, day, year = inp.split('/')
        if len(year) == 2: year = '19%s'%year
        month = int(month)
        day = int(day)
        return '%s-%02d-%02d'%(year, month, day)

    # Mar-20-2009; Mar 20, 2009; March 20, 2009; Mar. 20, 2009; Mar 20 2009; 
    p2 = r'((?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z\s\.\-]*\d{1,2}\b[\s|\.|\-|,]*\d{2,4})'
    def f2(inp):
        inp = cleanup(inp)
        month, day, year = inp.split()
        day = int(day)
        month = all_months[month[:3]]
        return '%s-%s-%02d'%(year, month, day)

    # 20 Mar 2009; 20 March 2009; 20 Mar. 2009; 20 March, 2009 
    p3 = r'((?:\d{1,2}\b)(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z\s\.\-,]*\d{4})'
    def f3(inp):
        inp = cleanup(inp)
        day, month, year = inp.split()
        day = int(day)
        month = all_months[month[:3]]
        return '%s-%s-%02d'%(year, month, day)

    # Mar 20th, 2009; Mar 21st, 2009; Mar 22nd, 2009 
    p4 = r'((?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z\s\.\-]*\d{1,2}(th|st|nd)(?:\s|\.|\-|,)*\d{2,4})'
    def f4(inp):
        return f2(inp)

    # Feb 2009; Sep 2009; Oct 2010 
    p5 = r'((?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*\b[\s\.\-,]*\d{2,4})'
    def f5(inp):
        inp = f'01 {inp}'
        return f3(inp)

    # 6/2008; 12/2009 
    p6 = r'(\d{1,2}/\d{2,4})'
    def f6(inp):
        month, year = inp.split('/')
        inp = f'{month}/1/{year}'
        return f1(inp)

    # 2009; 2010
    p7 = r'(\d{4})'
    def f7(inp):
        inp = f'1/1/{inp}'
        return f1(inp)

    dates = []
    pats = [p1, p2, p3, p4, p5, p6, p7]
    fmts = [f1, f2, f3, f4, f5, f6, f7]

    for line in df:
        line = line.strip()
        date = None
        for idx, p in enumerate(pats):
            m = re.search(p, line)
            if m:
                date = m.group(1)
                func = fmts[idx]
                date = func(date)
                dates.append(date)
                break
        if not date:
            dates.append(None)
    dates = [(idx, date) for idx, date in enumerate(dates)]
    dates.sort(key=lambda x: x[1])
    indices = [x[0] for x in dates]
    indices = pd.Series(indices)
    return indices
    
    #indices = [9, 84, 2, 53, 28, 153, 474, 13, 129, 98, 111, 225, 31, 171, 191, 486, 335, 415, 36, 323, 405, 422, 375, 380, 345, 57, 481, 436, 104, 154, 162, 299, 402, 95, 73, 156, 108, 182, 332, 82, 351, 278, 214, 155, 223, 473, 49, 317, 11, 319, 40, 165, 418, 370, 382, 3, 50, 363, 219, 465, 237, 23, 342, 204, 258, 315, 27, 93, 17, 303, 488, 283, 395, 309, 419, 123, 19, 117, 232, 72, 189, 318, 369, 493, 148, 239, 105, 336, 6, 200, 81, 65, 164, 434, 313, 378, 495, 424, 398, 5, 254, 296, 75, 167, 21, 259, 499, 150, 347, 78, 340, 441, 267, 361, 221, 134, 466, 39, 197, 355, 430, 80, 246, 444, 85, 215, 263, 74, 403, 458, 16, 25, 127, 454, 70, 44, 59, 103, 112, 429, 179, 88, 358, 470, 205, 137, 294, 397, 295, 35, 438, 247, 209, 61, 107, 285, 175, 99, 455, 24, 275, 421, 48, 426, 489, 136, 30, 274, 10, 178, 1, 280, 447, 185, 228, 135, 69, 492, 199, 352, 8, 276, 230, 334, 96, 38, 368, 168, 261, 404, 29, 437, 423, 54, 284, 485, 68, 32, 349, 41, 63, 416, 130, 55, 116, 76, 462, 330, 37, 256, 390, 216, 174, 180, 476, 312, 265, 115, 71, 218, 202, 440, 385, 373, 210, 89, 149, 26, 7, 435, 177, 482, 157, 412, 22, 194, 14, 151, 233, 206, 245, 122, 94, 461, 226, 97, 91, 51, 33, 453, 67, 46, 322, 66, 399, 487, 138, 62, 211, 52, 269, 119, 100, 442, 143, 310, 301, 113, 298, 478, 272, 354, 0, 249, 192, 86, 172, 357, 331, 163, 300, 450, 477, 308, 196, 133, 47, 359, 64, 42, 409, 406, 193, 238, 483, 140, 311, 388, 56, 236, 372, 110, 248, 60, 181, 203, 169, 326, 90, 292, 142, 479, 4, 124, 131, 324, 121, 166, 468, 365, 213, 87, 353, 101, 333, 114, 459, 45, 338, 18, 222, 343, 20, 224, 12, 79, 387, 251, 120, 471, 77, 376, 327, 432, 384, 321, 212, 407, 266, 145, 201, 456, 260, 305, 329, 420, 392, 158, 190, 417, 443, 83, 374, 125, 457, 147, 159, 328, 195, 377, 367, 394, 494, 304, 446, 43, 128, 262, 102, 449, 184, 469, 452, 234, 362, 356, 144, 291, 484, 188, 414, 92, 241, 306, 350, 425, 281, 207, 126, 146, 302, 451, 498, 339, 250, 344, 346, 348, 496, 106, 118, 270, 433, 307, 173, 252, 314, 410, 490, 277, 391, 264, 325, 289, 160, 132, 341, 337, 428, 187, 445, 497, 183, 271, 396, 293, 400, 360, 297, 371, 491, 389, 386, 288, 379, 268, 472, 273, 287, 176, 448, 411, 408, 242, 364, 58, 170, 467, 15, 240, 316, 229, 217, 109, 227, 290, 460, 393, 282, 34, 220, 208, 243, 139, 320, 383, 244, 286, 480, 431, 279, 198, 381, 463, 366, 255, 439, 401, 475, 152, 257, 235, 464, 253, 141, 231, 427, 161, 186, 413]
    #return pd.Series(indices)  # Your answer here

In [7]:
date_sorter()

0        9
1       84
2        2
3       53
4       28
5      153
6      474
7       13
8      129
9       98
10     111
11     225
12      31
13     171
14     191
15     486
16     335
17     415
18      36
19     323
20     405
21     422
22     375
23     380
24     345
25      57
26     481
27     436
28     104
29     154
      ... 
470    220
471    208
472    243
473    139
474    320
475    383
476    244
477    286
478    480
479    431
480    279
481    198
482    381
483    463
484    366
485    255
486    439
487    401
488    475
489    152
490    257
491    235
492    464
493    253
494    141
495    231
496    427
497    161
498    186
499    413
Length: 500, dtype: int64